# Karlsruhe data

In [ ]:
from datenguidepy import get_regions

In [ ]:
regions = get_regions()

In [ ]:
KA_regions = regions.loc[regions.name.str.contains("Karlsruhe")]
KA_regions

overview of nuts levels can be found here: https://en.wikipedia.org/wiki/NUTS_statistical_regions_of_Germany

In [ ]:
KA_city_district_id = "08212"

In [ ]:
from datenguidepy import Query

In [ ]:
query = Query.region(KA_city_district_id)

### Add some statistics

First get some overview of available statistics and short descriptions

In [ ]:
from datenguidepy import get_statistics

In [ ]:
# return statistical descriptions in English
statistics = get_statistics(target_language = "en")
statistics.head()

In [ ]:
statistics.loc[statistics.short_description.str.contains("housing"), ["short_description"]]

`datenguidepy` provides convenience functionality to get an overview of available statistics:

In [ ]:
from datenguidepy import get_availability_summary 

In [ ]:
availability_result = get_availability_summary()
availability_KA = availability_result.loc[KA_city_district_id].copy()

In [ ]:
availability_KA.dropna().sort_values(by=["entries"], ascending=False)

example query of some statistics:

In [ ]:
year_range = list(range(1995, 2018))
stat1 = query.add_field("BETR01")
stat1.add_args({"year": year_range})
stat2 = query.add_field("ERW012")
stat2.add_args({"year": year_range})


In [ ]:
query.results(
    verbose_statistics = True,
    add_units = True,
)